In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
# !ls
# change working directory
import os
# %cd ../
os.chdir("drive/My Drive/EDYTE-2023")

Mounted at /content/drive/


In [ ]:
%pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 115.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.6 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23777 sha256=53b176eee216eb8c3f445c3db2d9c606381182008e87b03e9ca8a3f88c08fc47
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=7fd68f5d212f79bf5c35bb507d785990795c47ef3ba

In [ ]:
# Import libraries
import pandas as pd
from keybert import KeyBERT
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import radius_neighbors_graph

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Import data (process steps)
df = pd.read_csv("data/process-steps.csv")

In [ ]:
df.head()

,process_id,process_title_el,step_num_id,step_exit,step_child,step_title,step_official,step_implementation,step_duration_min,step_duration_max,step_description,step_previous_child_selector,step_previous_child,step_note,step_co_org
0,439993,Αποδεικτικό φορολογικής ενημερότητας,1,0,0,Φυσική ταυτοποίηση πολίτη στο ΚΕΠ,Υπάλληλος ΚΕΠ,Έλεγχος,PT1M,PT7M,Ο υπάλληλος του ΚΕΠ προβαίνει σε έλεγχο ταυτοπ...,NaN,NaN,NaN,NaN
1,439993,Αποδεικτικό φορολογικής ενημερότητας,2,0,0,Υποβολή αίτησης στο Πληροφοριακό σύστημα,Υπάλληλος ΚΕΠ,Ενέργεια μέσω λογισμικού,PT1M,PT2M,Ο υπάλληλος ΚΕΠ σε συνεργασία με τον πολίτη συ...,NaN,NaN,NaN,NaN
2,439993,Αποδεικτικό φορολογικής ενημερότητας,3,1,0,Εκτύπωση αποδεικτικού φορολογικής ενημερότητας,Υπάλληλος ΚΕΠ,Ενέργεια μέσω λογισμικού,PT1M,PT2M,Ο υπάλληλος ΚΕΠ εκτυπώνει και παραδίδει το απο...,NaN,NaN,NaN,NaN
3,937099,Άδεια διακίνησης χοιροειδών,1,0,0,Εκκίνηση διαδικασίας,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,937099,Άδεια διακίνησης χοιροειδών,2,0,1,Παραλαβή αιτήματος για διακίνηση χοιροειδών,Αρμόδια Διεύθυνση,Χειροκίνητη ενέργεια,P1D,P2D,Ο αιτών (κάτοχος των ζώων) που επιθυμεί να μετ...,1.0,1.0,Η αίτηση συνυπογράφεται από τον κάτοχο-αποστολ...,NaN


In [ ]:
docs_step = df[~df["step_title"].isnull()]["step_title"].values.tolist()
len(docs_step)

19931

In [ ]:
# 
kw_model = KeyBERT(model="paraphrase-multilingual-mpnet-base-v2")

In [ ]:
# Greek stop words
stop_words = stops = stopwords.words('greek')
stop_words += ["της", "τη", "του", "από", "προς"]

In [ ]:
keywords = kw_model.extract_keywords(docs_step[1], keyphrase_ngram_range=(1,2), stop_words=stop_words,)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['δι', 'ἀλλ'] not in stop_words.
  warnings.warn(


In [ ]:
keywords

[('αίτησης πληροφοριακό', 0.8783),
 ('υποβολή αίτησης', 0.8459),
 ('αίτησης', 0.7599),
 ('πληροφοριακό σύστημα', 0.6482),
 ('πληροφοριακό', 0.6134)]

In [ ]:
keywords = kw_model.extract_keywords(docs_step[0], keyphrase_ngram_range=(1,2), stop_words=stop_words, use_mmr=True, diversity=0.5, top_n=3)
keywords

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['δι', 'ἀλλ'] not in stop_words.
  warnings.warn(


[('ταυτοποίηση πολίτη', 0.9239),
 ('πολίτη κεπ', 0.7788),
 ('φυσική ταυτοποίηση', 0.667)]

In [ ]:
keywords = kw_model.extract_keywords(docs_step, keyphrase_ngram_range=(1,3), stop_words=stop_words, use_maxsum=True, nr_candidates=20, top_n=3)
keywords

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['δι', 'ἀλλ'] not in stop_words.
  warnings.warn(


[[('φυσική', 0.2825), ('κεπ', 0.3635), ('ταυτοποίηση πολίτη', 0.9239)],
 [('σύστημα', 0.4792), ('πληροφοριακό', 0.6134), ('υποβολή αίτησης', 0.8459)],
 [('εκτύπωση', 0.2818),
  ('αποδεικτικού', 0.5207),
  ('φορολογικής ενημερότητας', 0.9161)],
 [('διαδικασίας', 0.8092),
  ('εκκίνηση', 0.8173),
  ('εκκίνηση διαδικασίας', 0.999)],
 [('διακίνηση', 0.4506),
  ('παραλαβή αιτήματος', 0.562),
  ('χοιροειδών', 0.7703)],
 [('διακίνηση', 0.3613),
  ('παραλαβή ηλεκτρονικού αιτήματος', 0.5741),
  ('χοιροειδών', 0.6765)],
 [('πρωτοκόλληση', 0.5096),
  ('αίτησης', 0.8767),
  ('πρωτοκόλληση αίτησης', 0.9595)],
 [('όρων', 0.2415),
  ('έλεγχος υγειονομικών όρων', 0.5873),
  ('όρων διακίνησης χοιροειδών', 0.853)],
 [('υγειονομικού', 0.4436),
  ('πιστοποιητικού άδειας διακίνησης', 0.5541),
  ('χοιροειδών', 0.678)],
 [('διαδικασίας', 0.8092),
  ('εκκίνηση', 0.8173),
  ('εκκίνηση διαδικασίας', 0.999)],
 [('υπεύθυνης', 0.525), ('δήλωσης', 0.6802), ('παραλαβή αίτησης', 0.855)],
 [('δικαιολογητικών', 0.3879),

In [ ]:
# Replace empty keywords with the raw text
keywords_tmp = [el if el else [(docs_step[i], 0.9999)] for i, el in enumerate(keywords)]

In [ ]:
# q1 = [docs_step[i] for i, el in enumerate(keywords_tmp) if el[0][1] == 0.9999]
# q1

In [ ]:
"kobebryant".split()

['kobebryant']

In [ ]:
# Flatten
flattened_keywords = [word for doc_keywords in keywords_tmp for item in doc_keywords for word in item[0].split()]
flattened_keywords

In [ ]:
doc_embeddings, word_embeddings = kw_model.extract_embeddings(flattened_keywords)

In [ ]:
connectivity = radius_neighbors_graph(
                doc_embeddings,
                radius=0.4,
                metric="euclidean",
                mode="connectivity"
)

agglClust = AgglomerativeClustering(
              n_clusters=None,
              distance_threshold=0.2,
              connectivity=connectivity,
              affinity="euclidean",
              linkage="average").fit(doc_embeddings)

labels = agglClust.labels_
